In [1]:
import os, ee, pickle
import geopandas as gpd

In [2]:
ee.Authenticate()
#ee.Initialize()

True

In [6]:
landsat8_collection = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filterDate('2022-07-01', '2022-07-31')

EEException: Not signed up for Earth Engine or project is not registered. Visit https://developers.google.com/earth-engine/guides/access

In [ ]:
shapefile = gpd.read_file("files/AllPossibleMeadows_2024-02-12.shp")

In [ ]:
import geemap
import ipyleaflet
import time, math
import ipywidgets as widgets

In [ ]:
ids = shapefile.ID

def getIdx():
    for idx in ids:
        yield idx

myIds = getIdx()
i = -1

In [ ]:
def zoom_level(area):
    ''' zoom_level ranges from 10 (largest of 369.83082 km2) to 19 (smallest of 0.0007 km2) for all polygons
     Each zoom-out approximately quadruples the area viewed (hence log 2)
     calculate deviation or zoom-out extent from 19 '''
    tradeoff = math.log2(area/0.0007)
    return (19 - round(tradeoff/2))

In [ ]:
toDel = []
toEdit = []
toPres = []

# pre-define map (map would be closed after each iteration) and dropdown object (to check selections)
Map = geemap.Map()
dropdown = widgets.Dropdown(options=["Preserve", "Delete", "Edit later"], style={"description_width": "initial"}, 
                            value=None, description="Take action (Meadow ID = 0): ")

In [ ]:
# Load previously saved data if such was saved

try:
    f = open('files/vars.pckl', 'rb')
    toDel, toEdit, toPres, i = pickle.load(f)
    f.close()
    ids = ids[i+1:]
except:
    pass

In [ ]:
idx = next(myIds)
row = shapefile[shapefile.ID == idx].iloc[0]
feature = row.geometry
lon, lat = feature.centroid.coords[0]

Map.close()
Map = geemap.Map(center=[lat, lon], zoom=zoom_level(row.Area_km2))
Map.add_basemap("SATELLITE")

gdf_selected = gpd.GeoDataFrame(geometry=[feature])
geo_data = ipyleaflet.GeoData(geo_dataframe=gdf_selected, style={'color': 'red', 'fillOpacity':0.01})
dropdown = widgets.Dropdown(options=["Preserve", "Delete", "Edit later"], style={"description_width": "initial"}, 
                            value=None, description="Take action (Meadow ID = " + str(int(idx)) + "):")
output_control = ipyleaflet.WidgetControl(widget=dropdown, position="topright")

Map.add_layer(geo_data)
Map.add_control(output_control)
i += 1
display(Map)

In [ ]:
# appends indices of choices made in map to lists for future processing
if dropdown.value == "Delete":
    toDel.append(i)
elif dropdown.value == "Edit later":
    toEdit.append(i)
elif dropdown.value == "Preserve":
    toPres.append(i)
else:
    print("Please make a selection for the above displayed meadow")

if (len(toDel) + len(toEdit) + len(toPres)) == len(ids):
    print("Meadow iteration is complete!")

In [ ]:
print("Latest meadow ID:", idx, end='\n\n')
print("Indexes to be deleted:", toDel, end='\n\n\n')
print("Meadow IDs to be edited later:", toEdit, end='\n\n\n')
print("Preserving IDs:", toPres)

In [ ]:
# Save your variables here

f = open('files/vars.pckl', 'wb')
pickle.dump([toDel, toEdit, toPres, i], f)
f.close()

In [ ]:
# delete non-meadows (if any were marked for deletion) and write data to new shapefile
if toDel:
    shapefile.drop(index=toDel, inplace=True)
shapefile.to_file('files/Real_Meadows.shp', driver='ESRI Shapefile')

# add column representing those to be edited (if any were marked for editing) and write to csv
if toEdit:
    shapefile['Action'] = None
    shapefile.loc[toEdit, 'Action'] = 'Edit'
cols = list(shapefile.columns)
cols.remove('geometry')
shapefile.loc[toEdit, cols].to_csv('csv/Real_meadows.csv', index=False)